In [338]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction
from heapq import nlargest
from functools import reduce
from operator import add
from scipy.spatial import distance
from numpy import array
import sklearn.metrics as metrics
from scipy.linalg import svd
import copy
import scipy.spatial.distance as distance
import Levenshtein as lev
import heapq
import random
import numpy
import time
import string
import json
import re

In [261]:
global recipes_before_removing_element
global flattened_recipes_set
global unique_ingredients
global unique_ingredients_after_removing_hidden

f = open('train.json')
data = json.load(f)

recipe_list = []

for i in data:
    #split with comma to get the cuisine.
    split1=str(i).split(',')
    #clean the strings with split and regex
    split2 = split1[1].split(':')
    cuisine = re.sub(r'[.\W]','',split2[1])
    #get the recipe id
    recipe_split = split1[0].split(': ')
    recipe_id=recipe_split[1]
    #keep only the pepperoni guys
    if cuisine=='italian':
        #splitting with [ because ingredients have the only list in the string
        split3=str(i).split('[')
        #final cut
        ingredients=split3[1].split(',')
        #removing ' from the strings
        ingredients = [s.translate(str.maketrans('','',string.punctuation)) for s in ingredients]
        if len(ingredients)>2:
            recipe_list.append(ingredients)
            
            
#remove whitespaces
for i,obj in enumerate(recipe_list):
    recipe_list[i] = [x.strip() for x in recipe_list[i]]

print(len(recipe_list))            
#make the 2d list 1d so we can count occurences easily
flattened_recipes_set = copy.deepcopy(recipe_list)
flattened_recipes_set = reduce(lambda x,y :x+y ,flattened_recipes_set)


#find the unique ingredients list of all the italian dataset
unique_ingredients = list(numpy.unique(numpy.array(flattened_recipes_set)))
print(len(unique_ingredients))            

#randomize the recipes
random.shuffle(recipe_list)


#get the first 1000 randomized items
set_of_recipes = recipe_list[:1000]


recipes_before_removing_element = copy.deepcopy(set_of_recipes)

hidden_items_to_remove = []

#remove 1 random ingredient from every recipes
for i,obj in enumerate(set_of_recipes):
    #randomise the list and remove the first element
    random.shuffle(set_of_recipes[i])
    hidden_items_to_remove.append(set_of_recipes[i][0])
    set_of_recipes[i].pop(0)


recipes=copy.deepcopy(set_of_recipes)
print(len(hidden_items_to_remove))
#unique ingredients have all the items except the hidden FOR ALL ITALIAN RECIPES
unique_ingredients_without_hidden = list(set(unique_ingredients) - set(hidden_items_to_remove))



7801
2932
1000


In [295]:
vectorized_matrix_m =[]
for i,obj in enumerate(recipe_list):
    vector = [0] * len(unique_ingredients_without_hidden)
    current_recipe = recipe_list[i]
    for j,obj in enumerate(current_recipe):
        if current_recipe[j] in unique_ingredients_without_hidden:
            vector[unique_ingredients_without_hidden.index(current_recipe[j])] = 1
    vectorized_matrix_m.append(vector)
print(len(vectorized_matrix_m))
print(len(vectorized_matrix_m[0]))

7801
2543


In [294]:
vectorized_matrix_r =[]
for i,obj in enumerate(recipes):
    vector = [0] * len(unique_ingredients_without_hidden)
    current_recipe = recipes[i]
    for j,obj in enumerate(current_recipe):
        if current_recipe[j] in unique_ingredients_without_hidden:
            vector[unique_ingredients_without_hidden.index(current_recipe[j])] = 1
    vectorized_matrix_r.append(vector)
print(len(vectorized_matrix_r))
print(len(vectorized_matrix_r[0]))

1000
2543


In [269]:
# Popular_without_recipes_ingredients is a dict but without the ingredients 
# of the current recipe of the loop.

score_list = []
#popular is a dictionary with : (ingredient,times igredient used in all the recipes)
popular = dict((x,flattened_recipes_set.count(x)) for x in set(flattened_recipes_set))
for i in recipes:
    popular_without_recipes_ingredients = list(filter(lambda x: x in i, popular))
    max_value = max(popular_without_recipes_ingredients)
    score_list.append(max_value)
most_popular_score_list = score_list

In [321]:

vectorized_matrix_r = numpy.array(vectorized_matrix_r)
vectorized_matrix_m = numpy.array(vectorized_matrix_m)

D = metrics.pairwise.pairwise_distances(vectorized_matrix_r, vectorized_matrix_m, metric='jaccard')

print(D)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1735: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


[[0. 1. 1. ... 1. 1. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 [1. 1. 0. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [350]:
#getting the length similar indexes for the recipes

lists_with_scores_for_every_n = []
for length in range(1,100):
    
    most_similar = []
    scores_ucf = []
    most_similar_values = []
    
    for i,obj in enumerate(recipes):



        #the Jaccard similarity of an item with itself is always 1.
        #we are going to get the n+1 largest values so we can pop out
        #the one with itself.

        #gives the indexes of the n+1 largest values of jaccard
        most_similar = heapq.nsmallest(length+1, range(len(a)), D[i].take)
        most_similar.pop(0)

        #gives the values of jaccard of the n+1 largest values
        jaccard_values = heapq.nsmallest(length+1, D[i])
        jaccard_values.pop(0)      
        scores_1=[]

        ingredients_without_current = list(set(unique_ingredients_without_hidden) - set(recipes[i]))

        for j,obj in enumerate(unique_ingredients_without_hidden):
            if unique_ingredients_without_hidden[j] in recipes[i]:
                scores_1.append(0)
            else:
                index_of_ingredient = unique_ingredients_without_hidden.index(unique_ingredients_without_hidden[j])

                jaccard_dot_matrix_sum = 0
                jaccard_sum = 0

                for k in range(length):
                    m = most_similar[k]
                    jaccard_dot_matrix_sum += jaccard_values[k] * vectorized_matrix_m[m][index_of_ingredient]
                    jaccard_sum += jaccard_values[k]

                if jaccard_sum != 0:    
                    scores_1.append(jaccard_dot_matrix_sum/jaccard_sum)
                else:
                    scores_1.append(0) 
        scores_ucf.append(scores_1)
    lists_with_scores_for_every_n.append(scores_ucf)

In [388]:
lists_of_precisions = []
for length in range(0,99):
    found_sum = 0
    for j,obj in enumerate(recipes):
        hidden_element = list(set(recipes_before_removing_element[j]) - set(recipes[j]))
        current_lengths_scores = lists_with_scores_for_every_n[length]

        #for K=1 we just want the maximum value
        max_value = max(current_lengths_scores[j])
        #print(max_value)
        max_index = current_lengths_scores[j].index(max_value)

        ingredient_proposed_by_the_algorithm = unique_ingredients_without_hidden[max_index]

        Ratio = lev.ratio(hidden_element[0].lower(),ingredient_proposed_by_the_algorithm.lower())
        if Ratio>0.5:
#             print("hidden element:",hidden_element[0])
#             print("found:",ingredient_proposed_by_the_algorithm)
#             print("++++++++")
            found_sum += 1
#     print("found sum:",found_sum)
#     print(found_sum/len(recipes))
#     print("------------")
    lists_of_precisions.append(found_sum/len(recipes))
print(lists_of_precisions)
print("best is at:",lists_of_precisions.index(max(lists_of_precisions)))

[0.01, 0.015, 0.017, 0.018, 0.025, 0.023, 0.028, 0.023, 0.025, 0.028, 0.022, 0.021, 0.022, 0.023, 0.02, 0.022, 0.023, 0.025, 0.024, 0.023, 0.027, 0.028, 0.026, 0.031, 0.031, 0.029, 0.026, 0.026, 0.028, 0.025, 0.026, 0.026, 0.023, 0.023, 0.023, 0.025, 0.025, 0.022, 0.02, 0.022, 0.02, 0.019, 0.017, 0.02, 0.021, 0.021, 0.02, 0.021, 0.02, 0.021, 0.021, 0.018, 0.015, 0.015, 0.017, 0.018, 0.017, 0.015, 0.013, 0.015, 0.014, 0.016, 0.016, 0.016, 0.015, 0.016, 0.015, 0.015, 0.017, 0.016, 0.015, 0.015, 0.014, 0.012, 0.014, 0.015, 0.016, 0.019, 0.019, 0.019, 0.017, 0.017, 0.018, 0.018, 0.019, 0.019, 0.019, 0.021, 0.019, 0.02, 0.021, 0.021, 0.019, 0.018, 0.019, 0.019, 0.02, 0.019, 0.019]
best is at: 23


In [490]:
#jaccard of each ingredient with the other
Distance_ingredients = []

for i in unique_ingredients_without_hidden:
    distance_ingredient_current = []
    for j in unique_ingredients_without_hidden:
        s1 = set(i)
        s2 = set(j)
        if len(s1.union(s2))==0:
            distance_ingredient_current.append(0)
        else:
            distance_ingredient_current.append(len(s1.intersection(s2)) / len(s1.union(s2)))
    Distance_ingredients.append(distance_ingredient_current)

Distance_ingredients = numpy.array(Distance_ingredients)


In [510]:
lists_with_scores_for_every_n_icf = []

for length in range(1,100):
    print("now checking",length)
    unique_most_similar=[]
    unique_j_values=[]
    
    for i,obj in enumerate(unique_ingredients_without_hidden):
        #gives the indexes of the n+1 largest values of jaccard
        most_similar = heapq.nlargest(length+1, range(len(Distance_ingredients[i])), Distance_ingredients[i].take)
        most_similar.pop(0)
        unique_most_similar.append(most_similar)

        #gives the values of jaccard of the n+1 largest values
        jaccard_values = heapq.nlargest(length+1, Distance_ingredients[i])
        jaccard_values.pop(0)
        unique_j_values.append(jaccard_values)


    most_similar = []
    jaccard_values = []
    scores_icf = []

    for i,obj in enumerate(recipes):

        ingredients_without_current = list(set(unique_ingredients_without_hidden) - set(recipes[i]))
        scores_1=[]
        for j,obj in enumerate(unique_ingredients_without_hidden):
            if unique_ingredients_without_hidden[j] in recipes[i]:
                scores_1.append(0)
            else:

                most_similar = unique_most_similar[j]
                jaccard_values = unique_j_values[j]
                jaccard_dot_matrix_sum = 0
                jaccard_sum = 0

                for k in range(length):
                    m = most_similar[k]
                    #in vectorized_m i dont need to find the index
                    #because the first 1000 recipes corresponds to the first 1000 recipes of the
                    #recipe_list that the matrix was built. it was randomized before so it doesnt matter
                    #that these are the first 1000
                    jaccard_dot_matrix_sum += jaccard_values[k] * vectorized_matrix_m[i][m]
                    jaccard_sum += jaccard_values[k]

                if jaccard_sum != 0:    
                    scores_1.append(jaccard_dot_matrix_sum/jaccard_sum)
                else:
                    scores_1.append(0) 
        scores_icf.append(scores_1)
    lists_with_scores_for_every_n_icf.append(scores_icf)

now checking 1
now checking 2
now checking 3
now checking 4
now checking 5
now checking 6
now checking 7
now checking 8
now checking 9
now checking 10
now checking 11
now checking 12
now checking 13
now checking 14
now checking 15
now checking 16
now checking 17
now checking 18
now checking 19
now checking 20
now checking 21
now checking 22
now checking 23
now checking 24
now checking 25
now checking 26
now checking 27
now checking 28
now checking 29
now checking 30
now checking 31
now checking 32
now checking 33
now checking 34
now checking 35
now checking 36
now checking 37
now checking 38
now checking 39
now checking 40
now checking 41
now checking 42
now checking 43
now checking 44
now checking 45
now checking 46
now checking 47
now checking 48
now checking 49
now checking 50
now checking 51
now checking 52
now checking 53
now checking 54
now checking 55
now checking 56
now checking 57
now checking 58
now checking 59
now checking 60
now checking 61
now checking 62
now checking 63
n

In [511]:
found_sum = 0
for j,obj in enumerate(recipes):
    hidden_element = list(set(recipes_before_removing_element[j]) - set(recipes[j]))
    #current_lengths_scores = lists_with_scores_for_every_n[length]

    #for K=1 we just want the maximum value
    max_value = max(scores_icf[j])
    #print(max_value)
    max_index = scores_icf[j].index(max_value)

    ingredient_proposed_by_the_algorithm = unique_ingredients_without_hidden[max_index]

    Ratio = lev.ratio(hidden_element[0].lower(),ingredient_proposed_by_the_algorithm.lower())
    if Ratio>0.5:
#         print("hidden element:",hidden_element[0])
#         print("found:",ingredient_proposed_by_the_algorithm)
#         print("++++++++")
        found_sum += 1

print("found sum:",found_sum)
print(found_sum/len(recipes))
#lists_of_precisions.append(found_sum/len(recipes))



found sum: 19
0.019
